# Polarization and EIS at different currents

In this example the [Zahner DC Sequencer](https://doc.zahner.de/manuals/sequencer.pdf) is used and EIS is measured.
With the DC Sequencer, OCP is measured first and then constant current is output. The constant current phase is followed by an impedance spectra. This example can be used as a template when measuring at different current densities, e.g. fuel cells.

The feature of this script is that the [Jinja2](https://jinja.palletsprojects.com/en/3.1.x/) template engine is used to create the sequence files with different currents, which are then executed.

For the DC Sequencer alone there is a separate [example](https://github.com/Zahner-elektrik/Thales-Remote-Python/blob/main/Examples/DCSequencer/DCSequencer.ipynb).

In [1]:
import os
import jinja2
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import (
    PotentiostatMode,
    ScanDirection,
    ThalesRemoteScriptWrapper,
    FileNaming,
    ScanStrategy,
)

# Function to generate the seq sequence files

There is a sequence file as template with the name *ocp_constant_current_template.txt*, this file contains the placeholders, which are then filled with Jinja2. For example, the placeholders are: `\PYVAR{dc_time}`. The function `fillTemplateFile()` is then passed the parameters as keyworded variables (kwargs) for example: `dc_time=2.0`.

Extract from the sequence file:
```
  start_cycle	
    samples(10)
    ocp(TIM2)
    hold_cur(CUR1,TIM1)
  end_cycle
    
  start_variables
    TIM1=\PYVAR{dc_time}
    TIM2=\PYVAR{ocp_time}
    CUR1=\PYVAR{dc_current}
  end_variables
```

At 10 Hz the measurements are performed, first an OCP phase and then a constant current phase.

When the template is filled, the complete file is written according to the corresponding path with the filename.

In [2]:
def fillTemplateFile(templateFile: str, outputFile: str, **kwargs):
    """
    Function to fill in template files.

    For example, the following placeholders must be in the passed template:
        `\PYVAR{dc_time}`

    Then the value for the template must be passed as an additional parameter as keyworded variable (kwargs):
        `dc_time=2.0`

    :param templateFile: path of the file containing the templates.
    :param outputFile: path to the file containing the completed templates.
    :param **kwargs: template parameters which should be filled in as keyworded variable.
    """
    latex_jinja_env = jinja2.Environment(
        variable_start_string="\PYVAR{",
        variable_end_string="}",
        trim_blocks=True,
        autoescape=False,
        keep_trailing_newline=True,
        newline_sequence="\r\n",
        loader=jinja2.FileSystemLoader(os.path.abspath(".")),
    )

    template = latex_jinja_env.get_template(templateFile)

    fileString = template.render(**kwargs)

    fileBytes = fileString.encode("utf-8")
    with open(outputFile, "wb") as f:
        f.write(fileBytes)

    return

# Initialize the Measurement

The list `currentSteps` defines the DC currents at which DC is measured and at which impedance is measured.
The variable `amplitude` specifies the amplitude which is used for all currents.

Afterwards the connection to the already running Thales/Term software is established.

In [3]:
currentSteps = [0, 0.5, 1, 1.5, 2]
amplitude = 0.05

zenniumConnection = ThalesRemoteConnection()
zenniumConnection.connectToTerm("localhost")
zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
zahnerZennium.forceThalesIntoRemoteScript()

devel version


'128,ScriptRemote,5,6,0,0'

# Perform the Measurements

A for loop is used to iterate over the different current points. For each current a new seq-file is created with the function `fillTemplateFile()` from the template file.

Then the sequence written to number 9 is performed and the EIS measurement is parameterized and done in the same way. For the DC measurement and the EIS, a file is saved, which has the current in mA in the name, since there must not be any "," or "." in the filename.

In [4]:
for current in currentSteps:
    print(f"Step: {current}")
    """
    DC Sequence
    """
    fillTemplateFile(
        templateFile=r"ocp_constant_current_template.txt",
        outputFile=r"C:\THALES\script\sequencer\sequences\sequence09.seq",
        ocp_time=10,
        dc_time=20,
        dc_current=current,
    )

    filename = f"{int(current*1000)}ma_current"

    zahnerZennium.disableSequenceAcqGlobal()
    zahnerZennium.setSequenceNaming(FileNaming.INDIVIDUAL)
    zahnerZennium.setSequenceOutputPath(r"C:\THALES\temp")
    zahnerZennium.setSequenceOutputFileName(filename)
    zahnerZennium.selectSequence(9)
    zahnerZennium.runSequence()

    """
    EIS
    """
    zahnerZennium.setEISNaming(FileNaming.INDIVIDUAL)
    zahnerZennium.setEISOutputPath(r"C:\THALES\temp")
    zahnerZennium.setEISOutputFileName(filename)
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_GALVANOSTATIC)
    zahnerZennium.setAmplitude(amplitude)
    zahnerZennium.setCurrent(current)
    zahnerZennium.setLowerFrequencyLimit(10)
    zahnerZennium.setStartFrequency(1000)
    zahnerZennium.setUpperFrequencyLimit(10000)
    zahnerZennium.setLowerNumberOfPeriods(5)
    zahnerZennium.setLowerStepsPerDecade(2)
    zahnerZennium.setUpperNumberOfPeriods(20)
    zahnerZennium.setUpperStepsPerDecade(5)
    zahnerZennium.setScanDirection(ScanDirection.START_TO_MAX)
    zahnerZennium.setScanStrategy(ScanStrategy.SINGLE_SINE)

    zahnerZennium.measureEIS()

    zahnerZennium.setAmplitude(0)

zahnerZennium.disablePotentiostat()
zenniumConnection.disconnectFromTerm()

Step: 0
Step: 0.5
Step: 1
Step: 1.5
Step: 2
